# Geschwindigkeit Tests der Transpositionstabelle

In [ ]:
import os.path
css = ""
if os.path.isfile("style.html"):
    from IPython.core.display import HTML
    with open("style.html", "r") as file:
        css = file.read()
HTML(css)

Die Klasse `Cache` aus dem Notebook `nmm-cache` implementiert eine Transpositionstabelle die persistiert werden kann. Um zu ermitteln wie diese implementiert werden soll, wurde zwischen zwei Möglichkeiten abgewogen.

1. Zum einen steht die Verwendung einer externen Key-Value-Datenbank zur Auswahl. Diese wird in einem weiteren Container auf dem selben Computer ausgeführt, dadurch werden die Netzwerklatenzen minimiert.
2. Die zweite Möglichkeit ist das Speichern im Arbeitsspeicher in einem Python-Dictionary. Dieses kann anschließend exportiert und auf einen persistenten Datenträger geschrieben werden.

Um zu ermitteln welche Methode verwendet werden sollte, wird im Nachfolgenden die Geschwindigkeit der jeweiligen Methode getestet und bewertet.

## Vorbereitungen
Bevor die Möglichkeiten getestet werden können, müssen folgende Pakete geladen werden:
- `redis` implementiert die Kommunikation mit der Redis-Datenbank;
- `struct` übersetzt `float` Werte in `bytes`;
- `random` ermittelt Zufallswerte zum testen;
- `tqdm` zeigt den Fortschritt an.

In [ ]:
import redis
import struct
import random
from tqdm.notebook import tqdm

Des Weiteren wird die Klasse `CacheTest` angelegt, welche als Interface für die jeweilige Implementierung dient. Zwei Funktionen werden vorgegeben:

* `write` schreibt die Werte `value`, `alpha` und `beta` an dem Schlüssel `state` und `player` in den Cache;
* `read` ließt die mit `write` gespeicherten Werte aus oder gibt `None` zurück, falls der Cache diese nicht beinhaltet.

In [ ]:
class CacheTest():
    def write(self, state: int, player: bool, limit: int, value: float, alpha: float, beta: float) -> None:
        pass
    
    def read(self, state: int, player: bool, limit: int) -> (float, float, float):
        """Value, Alpha, Beta"""
        pass


## Python-Dictionary
Der native Python Cache besteht aus einem einfachen Python-Dictionary in welches die Key-Value-Paare gespeichert werden.

In [ ]:
class PythonCache(CacheTest):
    def __init__(self):
        self.cache = {}
        pass
    
    def write(self, state: int, player: bool, limit: int, value: float, alpha: float, beta: float) -> None:
        key = state.to_bytes(8,'big') + player.to_bytes(1,'big') + limit.to_bytes(1,'big')
        value = struct.pack("d", value) + struct.pack("d",alpha) + struct.pack("d",beta)
        self.cache[key] = value
    
    def read(self, state: int, player: bool, limit: int) -> (float,float,float):
        """Value, Alpha, Beta"""
        key = state.to_bytes(8,'big') + player.to_bytes(1,'big') + limit.to_bytes(1,'big')
        value = self.cache.get(key)
        return (
            struct.unpack("d", value[:8])[0],
            struct.unpack("d", value[8:16])[0],
            struct.unpack("d", value[16:24])[0],
        ) if value else None

## Redis

Der Redis-Cache implementiert eine Transpositionstabelle auf basierend auf der Key-Value-Datenbank *Redis*. Die Übersetzung der Werte zu Byte-Arrays erfolgt äquivalent zu dem Python-Cache. Die Werte werden danach mit Hilfe des `redis` Pakets in der lokalen Redis-Datenbank gespeichert.

In [ ]:
class RedisCache(CacheTest):
    r = redis.Redis(host='redis')
    def write(self, state: int, player: bool, limit: int, value: float, alpha: float, beta: float) -> None:
        key = state.to_bytes(8,'big') + player.to_bytes(1,'big') + limit.to_bytes(1,'big')
        value = struct.pack("d",value) + struct.pack("d",alpha) + struct.pack("d",beta)
        self.r.set(key,value)

    def read(self, state: int, player: bool, limit: int):
        key = state.to_bytes(8,'big') + player.to_bytes(1,'big') + limit.to_bytes(1,'big')
        value = self.r.get(key)
        return (
            struct.unpack("d", value[:8])[0],
            struct.unpack("d", value[8:16])[0],
            struct.unpack("d", value[16:24])[0],
        ) if value else None

## Test

Als Vorbereitung des Tests werden zunächst Instanzen der einzelnen `CacheTest` Implementierungen erstellt. Außerdem werden Einstellungen für die Tests vorgenommen:

* `count` ist die Anzahl der Elemente, die in die Caches geschrieben werden sollen;
* `start` bis `end` ist das Intervall in dem Zustände generiert werden. Ein Zustand ist in diesem Test jedeglich eine Zahl.

In [ ]:
pc = PythonCache()
rc = RedisCache()

count =     1_000_000
start = 1_000_000_000
end   = start+count

### Test der Schreibgeschwindigkeit

Durch die Einstellung `count` gesteuert werden zufällige Werte in den Caches gespeichert. In diesem Fall sind es eine Millionen Einträge.

In [ ]:
%%time
print('Python Cache')
for state in tqdm(range(start, end)):
    for player in [True, False]:
        for limit in range(4):
            pc.write(state, player, limit, random.random(), random.random(), -random.random())

In [ ]:
%%time
print('Redis Cache')
for state in tqdm(range(start, end)):
    for player in [True, False]:
        for limit in range(4):
            rc.write(state, player, limit, random.random(), random.random(), -random.random())

Im Vergleich zu dem `PythonCache` ist der `RedisCache` bei dem Schreibvorgang der Einträge etwa um den Faktor $90$ langsamer.

### Testen der Lesegeschwindigkeit
Bei dem Test der Lesegeschwindigkeit wird gemessen, wie schnell die Werte für zufällig ermittelte Zustände zurück gegeben werden können. Dabei wird auf die oben generierten Einträge zurück gegriffen.

In [ ]:
%%time
valid = 0
for count in tqdm(range(count)):
    state = random.randint(start, end-1)
    limit = random.randint(0, 3)
    player = random.choice([True, False])
    result = pc.read(state, player, limit)
    if result is None:
        print(state, player, limit)
    else:
        valid += 1
print(valid)

In [ ]:
%%time
valid = 0
for count in tqdm(range(count)):
    state = random.randint(start, end-1)
    limit = random.randint(0, 3)
    player = random.choice([True, False])
    result = rc.read(state, player, limit)
    if result is None:
        print(state, player, limit)
    else:
        valid += 1
print(valid)

Während die Lesegeschwindigkeit beim `RedisCache` deutlich höher als die Schreibgeschwindigkeit ist, ist diese dennoch um etwa den Faktor $4$ langsamer als die des `PythonCache`s.

## Resumee
Sowohl die Lese- als auch die Schreibgeschwindigkeit ist beim `PythonCache` deutlich höher als beim `RedisCache`. Wobei die Geschwindigkeitsdifferenz beim Schreiben stärker ausfällt als beim Lesen. Dies ist höchst wahrscheinlich auf den Overhead der Verbindung zwischen Datenbank und dem Python-Programm zurückzuführen.

Aus diesem Grund wurde sich dafür entschieden, den Cache als Python-Dictionary zu implementieren und anschließend als Datei auf dem Datenträger zu persistieren. Hierbei wird in Kauf genommen, dass die wachsende Größe des Caches zu einer starken Auslastung des Arbeitsspeicher führen kann.